In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import imageDataGenerator
import pickle

In [1]:
data_path = r"C:\Users\Richa\OneDrive\Documents\Projects\Deployments\Sports_classification\data"
output_model = r"C:\Users\Richa\OneDrive\Documents\Projects\Deployments\Sports_classification\output_model"
output_lable_binarizer = r"C:\Users\Richa\OneDrive\Documents\Projects\Deployments\Sports_classification\output_lable_binarizer"
epoch=25

In [ ]:
Sports_Lable = set(["boxing","swimming","table_tennis"])
print("images are being uploaded.....")
path_to_images = list(paths.list_images(data_path))
data =[]
lables =[]

for image_path in path_to_images:
    lable = image_path.split(os.path.sep)[-2]
    if lable in Sports_Lable:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224))
        data.append(image)
        lables.append(lable)

In [ ]:
data = np.array(data)
lables = np.array(lables)
lb = LabelBinarizer()
lables = lb.fit_transform(lables)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, lables, test_size=0.25, stratify=lables, random_state=42)

In [ ]:
trainingAugmentation =  imageDataGenerator(
                                rotation_range = 30,
                                zoom_range = 0.15,
                                width_shift_range = 0.2,
                                height_shift_range = 0.2,
                                shear_range = 0.15,
                                horizontal_flip = True,
                                fill_mode = "nearest"
                                )
valiadationAugmentation = imageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainingAugmentation.mean = mean
valiadationAugmentation.mean = mean

In [ ]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Model

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(len(lb.classes_), activation="softmax")(head_model)
model = Model(inputs=base_model.input, outputs=head_model)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from keras.optimizers import SGD

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epoch)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
history = model.fit_generator(
    trainingAugmentation.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=valiadationAugmentation.flow(X_test, y_test),
    validation_steps=len(X_test) // 32,
    epochs=epoch
)

In [ ]:
model.save(output_model)
lbinarizer = open("video_classification_binarizer.pickel", "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()